# Simplify polygons and Query Load_Collection with different CRSs
The `load_collection` process takes a GeoJSON for spatial specification. GeoJSON dropped the possibility to specify crs, so it is always assumed to be WGS84 (EPSG:4326). This means that if we have a geometry or FeatureCollection that is e.g SWEREF99, we need to convert it first. This notebook will show you :

1. We load a massive WGS84 Polygon
2. We simplify it (to get an answer to compare with)
3. We asked chatgpt to translate it, pretending that we had SWEREF99 from the beginning
4. We translate the simplified SWEREF99 polygon to WGS84

In [1]:

import json
from shapely.geometry import Polygon, mapping
from typing import Dict  # This is for typing the GeoJSON which is essentially a dictionary
import geopandas as gpd
from shapely.geometry import Polygon
from icecream import ic


In [2]:

import io
import numpy as np

from shapely.geometry import shape, Polygon

import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

from pyproj import Transformer

plt.rcParams['figure.figsize'] = [20, 10]

In [3]:

def simplify_polygon_on_geometries(geojson: Dict) -> Dict:
    # Extract the original polygon from the GeoJSON
    assert(len(geojson['features']) ==1) # ITs just a demo so we keep it simple with ONE polygon
    
    original_polygon = Polygon(geojson['features'][0]['geometry']['coordinates'][0])
    tolerance = 0.0001  
    simplified_polygon = original_polygon.simplify(tolerance, preserve_topology=True)
    simplified_geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": geojson['features'][0]['properties'],
                "id": geojson['features'][0].get('id', None),  # Use .get() to avoid KeyError if 'id' does not exist
                "geometry": mapping(simplified_polygon)
            }
        ]
    }
    return simplified_geojson


def load_geometries(fname: str)->Dict:
    with open(fname,'r') as fd:
        data = json.loads(fd.read())
        return data

In [4]:
geometries_wgs84 = load_geometries('big_geometry.json')
simplified_geoms_wgs84 = simplify_polygon_on_geometries(geometries_wgs84)
# Let's simplify the polygon
simplified_geoms_wgs84

FileNotFoundError: [Errno 2] No such file or directory: 'big_geometry.json'

Let's pretend that we had above in EPSG:3006 (SWEREF99)

In [5]:
# This is what odc expects and what our result should be
geoms_wgs84 = {'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'avverkningsdatum': '2020-06-12'},
   'id': 0,
   'geometry': {'type': 'Polygon',
    'coordinates': (((17.665270930775755, 59.995379244747966),
      (17.67062992671201, 59.99726841066224),
      (17.6706665367172, 59.99731246487363),
      (17.66865618054931, 59.998562597273036),
      (17.667887659980714, 59.998350223136455),
      (17.667531275365462, 59.99793238650041),
      (17.66668280978647, 59.99774021838905),
      (17.667259639988757, 59.99742845749122),
      (17.666987595091204, 59.99715474568522),
      (17.666593975117546, 59.99712641147649),
      (17.664064888468328, 59.998133391566974),
      (17.6637266530725, 59.99590200378276),
      (17.66490257827024, 59.995624959444285),
      (17.665270930775755, 59.995379244747966)),)}}]}
# Chatgpt converted to EPSG:3006
geoms_sweref99={
    'type': 'FeatureCollection',
    'features': [
        {
            'type': 'Feature',
            'properties': {'avverkningsdatum': '2020-06-12'},
            'id': 0,
            'geometry': {
                'type': 'Polygon',
                'coordinates': [
                    [
                        (5716940.96041046, 2687980.6815579724),
                        (5716966.282993964, 2688828.9467497403),
                        (5716970.740445865, 2688836.0488690934),
                        (5717230.718230454, 2688600.0323885423),
                        (5717235.143966627, 2688480.837941977),
                        (5717193.262969286, 2688412.163170612),
                        (5717203.96461507, 2688282.344769442),
                        (5717135.828371019, 2688352.037997783),
                        (5717110.08179383, 2688301.514763763),
                        (5717123.412121972, 2688243.8417277983),
                        (5717372.634078418, 2687923.1638540244),
                        (5717080.455702996, 2687781.0093295793),
                        (5716990.882917845, 2687938.13204107),
                        (5716940.96041046, 2687980.6815579724)
                    ]
                ]
            }
        }
    ]
}
# Converting the in-polygon to WGS84 from SWEREF99

gdf_sweref99 = gpd.GeoDataFrame.from_features(geoms_sweref99)
gdf_sweref99.set_crs(epsg=3006, inplace=True) # GeoJSON does not have this set anymore
gdf_wgs84 = gdf_sweref99.to_crs(epsg=4326)

ic(gdf_wgs84.to_json())

ic| gdf_wgs84.to_json(): ('{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", '
                          '"properties": {"avverkningsdatum": "2020-06-12"}, "geometry": {"type": '
                          '"Polygon", "coordinates": [[[59.9953792447479, 17.66527093077576], '
                          '[59.99726841066217, 17.67062992671202], [59.997312464873566, '
                          '17.670666536717206], [59.99856259727298, 17.66865618054932], '
                          '[59.99835022313639, 17.667887659980725], [59.997932386500345, '
                          '17.667531275365477], [59.997740218388984, 17.666682809786483], '
                          '[59.997428457491154, 17.667259639988767], [59.99715474568516, '
                          '17.66698759509121], [59.997126411476415, 17.666593975117564], '
                          '[59.99813339156691, 17.664064888468342], [59.995902003782696, '
                          '17.663726653072516], [59.9956249594442

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"avverkningsdatum": "2020-06-12"}, "geometry": {"type": "Polygon", "coordinates": [[[59.9953792447479, 17.66527093077576], [59.99726841066217, 17.67062992671202], [59.997312464873566, 17.670666536717206], [59.99856259727298, 17.66865618054932], [59.99835022313639, 17.667887659980725], [59.997932386500345, 17.667531275365477], [59.997740218388984, 17.666682809786483], [59.997428457491154, 17.667259639988767], [59.99715474568516, 17.66698759509121], [59.997126411476415, 17.666593975117564], [59.99813339156691, 17.664064888468342], [59.995902003782696, 17.663726653072516], [59.99562495944422, 17.664902578270254], [59.9953792447479, 17.66527093077576]]]}}]}'

Now you can use above in e.g load_collection